# Assignment 2

#### First Name: Clàudia
#### Last Name: Valverde Sanchez


## 1. Load Data from JSON

In [0]:
# Make sure to upload the corona_tweet_new.json file
df_twitter = spark.read.json("/FileStore/tables/corona_tweet_new.json")

In [0]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- follow_request_sent: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- following: string (nullable = true)
 |    |-- frie

In [0]:
### From the user nested col select the following cols only id_str,followers_count,friends_count and created at 
# (2 points)
from pyspark.sql.functions import col 

# Select the specific nested columns

df_twitter=df_twitter.select(
    col("created_at"),
    col("favorite_count"),
    col("hashtags"),
    col("id"),
    col("in_reply_to_status_id"),
    col("in_reply_to_user_id_str"),
    col("location"),
    col("reply_count"),
    col("retweet_count"),
    col("source"),
    col("user.id_str").alias("user_id_str"),
    col("user.followers_count").alias("user_followers_count"),
    col("user.friends_count").alias("user_friends_count"),
    col("user.created_at").alias("user_created_at")
)

# Show the selected columns
df_twitter.show()

+--------------------+--------------+--------------------+-------------------+---------------------+-----------------------+--------+-----------+-------------+--------------------+-------------------+--------------------+------------------+--------------------+
|          created_at|favorite_count|            hashtags|                 id|in_reply_to_status_id|in_reply_to_user_id_str|location|reply_count|retweet_count|              source|        user_id_str|user_followers_count|user_friends_count|     user_created_at|
+--------------------+--------------+--------------------+-------------------+---------------------+-----------------------+--------+-----------+-------------+--------------------+-------------------+--------------------+------------------+--------------------+
|Mon Apr 20 15:01:...|          5545|                  []|1252251164227362821|                 null|                   null|   India|       3460|         5477|<a href="https://...|           93612837|              

In [0]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)



In [0]:
# Print the total count of number of records in df_twitter(1 point)
df_twitter.count()

Out[8]: 15894

In [0]:
# Extract the source lable from source col by droping the anchor tab and save it as another col named extracted_source
# for example <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a> => Twitter Web App
# you can use "<a [^>]+>([^<]+)" as regualr expresion and the group would be 1 for this regular expression.
#(4 points)
from pyspark.sql.functions import regexp_extract, col

# Extract the source label from the source column and save it as another column named extracted_source
df_twitter = df_twitter.withColumn(
    "extracted_source",
    regexp_extract(col("source"), r'<a [^>]+>([^<]+)</a>', 1)
)
df_twitter.select(col('extracted_source'),col('source')).show()

+-------------------+--------------------+
|   extracted_source|              source|
+-------------------+--------------------+
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter Web Client|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
+----------

In [0]:
# Convert the DataFrame into RDD
rdd_twitter=df_twitter.rdd.map(tuple)

In [0]:
# Create a temporay table in memory with name as twitter (1 point)

#createOrReplaceTempView method. This method registers the DataFrame as a temporary table, which you can then query using SQL.
df_twitter.createOrReplaceTempView("twitter")


# to check
result = spark.sql("SELECT * FROM twitter LIMIT 10")
result.show()

+--------------------+--------------+--------+-------------------+---------------------+-----------------------+--------+-----------+-------------+--------------------+-------------------+--------------------+------------------+--------------------+-------------------+
|          created_at|favorite_count|hashtags|                 id|in_reply_to_status_id|in_reply_to_user_id_str|location|reply_count|retweet_count|              source|        user_id_str|user_followers_count|user_friends_count|     user_created_at|   extracted_source|
+--------------------+--------------+--------+-------------------+---------------------+-----------------------+--------+-----------+-------------+--------------------+-------------------+--------------------+------------------+--------------------+-------------------+
|Mon Apr 20 15:01:...|          5545|      []|1252251164227362821|                 null|                   null|   India|       3460|         5477|<a href="https://...|           93612837|  

## 2. Analyze Data

#### You will be writing code to find the answer to the questions listed below using Just RDD, Using spark SQL 

- Analyze using RDD 
- Analyze using Dataframe without temp table 
- Analyze using spark.sql with temp table


#### 2.1 Get total number of unique users (1 point for each type)

**My reasoning:** Selecting the user name column (user_id_str), count the number of unique (distinct) user names

In [0]:
# Using RDD

'''
Knowing user_id_str corresponds to column 10 of the original df, we select it and get the unique users
'''

# checking if I'm selecting the correct column
selected_column = rdd_twitter.map(lambda tweet: tweet[10]).take(10)
print("Sample of column user_id_str:")
for value in selected_column:
    print(value)
  

unique_users = rdd_twitter.map(lambda tweet: tweet[10]).distinct().count()
print(f"\nTotal number of unique users: {unique_users}")

Sample of column user_id_str:
93612837
346443880
1154311994
1244090006408327169
2781749028
1297207694
1247795796893814784
2258446142
549592791
816782950855491585

Total number of unique users: 14094


In [0]:
# Using DataFrame

# Get the total number of unique users
unique_users_count = df_twitter.select("user_id_str").distinct().count()

print(f"Total number of unique users: {unique_users_count}")

Total number of unique users: 14094


In [0]:
# Using spark.sql and the temporay table.

# Use Spark SQL to get the total number of unique users
unique_users_count = spark.sql("SELECT COUNT(DISTINCT user_id_str) AS unique_users FROM twitter").collect()[0]['unique_users']

print(f"Total number of unique users: {unique_users_count}")

Total number of unique users: 14094


#### 2.2 Get count of user who have more than 1 tweet in the data (2 points)


**My reasoning:** In order to tackle the exercise, first I need to know the number of tweets each user has done, after that, select only those users who have done more than 1 tweet and finally count how many fulfill all these conditions.


In [0]:
# Using RDD

# Map each tweet to the user ID
user_ids = rdd_twitter.map(lambda tweet: (tweet[10], 1)) # desired column is in position 10

# Count the occurrences of each user ID
user_tweet_counts = user_ids.countByKey()

# Filter users with more than one tweet
users_with_multiple_tweets = list(filter(lambda x: x[1] > 1, user_tweet_counts.items()))

# Count the number of filtered users
count_users_with_multiple_tweets = len(users_with_multiple_tweets)

print("Number of users who have more than one tweet:", count_users_with_multiple_tweets)


Number of users who have more than one tweet: 1016


In [0]:
# Using DataFrame

# Group by user_id_str and count the number of tweets for each user
count_users_with_multiple_tweets = df_twitter.groupby("user_id_str").count().filter("count>1").count()

print("Count of users who have more than one tweet:", count_users_with_multiple_tweets)

Count of users who have more than one tweet: 1016


In [0]:
# Using spark.sql and the temporay table.

# Count the number of tweets for each user and filter users with more than one tweet
sql_query = """
    SELECT COUNT(*)
    FROM (SELECT COUNT(user_id_str) AS tweet_count FROM twitter GROUP BY user_id_str)
    WHERE tweet_count > 1 
"""

# Execute the SQL query
users_with_multiple_tweets_sql = spark.sql(sql_query).collect()[0]['count(1)']

print("Count of users who have more than one tweet:", users_with_multiple_tweets_sql)

Count of users who have more than one tweet: 1016


#### 2.3 Get total number unique extracted_source (1 point each)

**My reasoning:** First filter by the column corresponding to extracted_source (14) and then count the number of unique (distinct) entries

In [0]:
# Using RDD

# checking if I'm selecting the correct column
selected_column = rdd_twitter.map(lambda tweet: tweet[14]).take(10)
print("Sample of column extracted_source:")
for value in selected_column:
    print(value)
  
unique_extracted_source = rdd_twitter.map(lambda tweet: tweet[14]).distinct().count()
print(f"\nTotal number of unique extracted_source: {unique_extracted_source}")

Sample of column extracted_source:
Twitter Web App
Twitter for Android
Twitter for Android
Twitter for Android
Twitter for Android
Twitter Web App
Twitter Web Client
Twitter for Android
Twitter for Android
Twitter for iPhone

Total number of unique extracted_source: 133


In [0]:
# Using DataFrame

# Get the total number of unique users
unique_extracted_source_count = df_twitter.select("extracted_source").distinct().count()

print(f"Total number of unique extracted_source: {unique_extracted_source_count}")

Total number of unique extracted_source: 133


In [0]:
# Using spark.sql and the temporay table.

# Get the total number of unique extracted_source
sql_query = """
    SELECT COUNT(DISTINCT extracted_source) AS unique_extracted_source 
    FROM twitter
"""
unique_extracted_source_count = spark.sql(sql_query).collect()[0]['unique_extracted_source']

print(f"Total number of unique extracted_source: {unique_extracted_source_count}")

Total number of unique extracted_source: 133


#### 2.4 Get top 5 most used extracted_source

**My reasoning:** In order to tackle the exercise, first I need to know how many times each extracted_source appears, after that, select the top 5 entries in number of extracted_source.
 

In [0]:
# Using RDD (5 points)

# Map each row of the DataFrame to extract the extracted_source value
rdd_extracted_sources = rdd_twitter.map(lambda row: row[14])

# Filter out null extracted_source values (not really necessary)
rdd_filtered_sources = rdd_extracted_sources.filter(lambda source: source is not None)

# Count the occurrences of each extracted_source
rdd_source_counts = rdd_filtered_sources.map(lambda source: (source, 1)).reduceByKey(lambda a, b: a + b)

# Sort the results in descending order by the count
rdd_sorted_sources = rdd_source_counts.sortBy(lambda x: x[1], ascending=False)

# Take the top 5 rows
top_5_sources = rdd_sorted_sources.take(5)

# Print the top 5 most used extracted_source values
for source, count in top_5_sources:
    print("extracted_source:", source, "source_count:", count)

extracted_source: Twitter for Android source_count: 6262
extracted_source: Twitter for iPhone source_count: 5698
extracted_source: Twitter Web App source_count: 2878
extracted_source: Twitter for iPad source_count: 428
extracted_source: Twitter Web Client source_count: 136


In [0]:
# Using DataFrame (2 points)

df_filtered_sources = df_twitter.groupby('extracted_source').count().sort("count", ascending = False).show(5)

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+
only showing top 5 rows



In [0]:
# Using spark.sql and the temporay table. (2 points)

# Get the top 5 most used extracted_source values
sql_query = """
    SELECT extracted_source, COUNT(*) AS source_count
    FROM twitter
    WHERE extracted_source IS NOT NULL
    GROUP BY extracted_source
    ORDER BY source_count DESC
    LIMIT 5
"""

# Execute the SQL query
top_5_sources_df = spark.sql(sql_query)

# Show the top 5 most used extracted_source values
top_5_sources_df.show()


+-------------------+------------+
|   extracted_source|source_count|
+-------------------+------------+
|Twitter for Android|        6262|
| Twitter for iPhone|        5698|
|    Twitter Web App|        2878|
|   Twitter for iPad|         428|
| Twitter Web Client|         136|
+-------------------+------------+



#### 2.5 Get count of distinct hashtags used (5 point each) 

**My reasoning:** From hashtags column (2), get the unique (distinct) hashtags and count how many there are.

Hashtags columns has a different structure than the rest of the column, each value is a kind of list type. So will have to use 'explode' or 'flatMap' on the column for the different procedures.

In [0]:
# Using RDD

# Explode the RDD to create one row per hashtag
rdd_exploded = rdd_twitter.flatMap(lambda x: x[2]) # col2 corresponds to hastags column

# Count the distinct hashtags
distinct_hashtags_count = rdd_exploded.distinct().count()

print("Distinct hashtags count:", distinct_hashtags_count)


Distinct hashtags count: 1215


In [0]:
# Using DataFrame
from pyspark.sql.functions import explode, col

# Explode the hashtags column to create one row per hashtag
df_exploded = df_twitter.select(explode(col('hashtags')))

# Count the distinct hashtags
distinct_hashtags_count = df_exploded.select('col').distinct().count()

print("Distinct hashtags count:", distinct_hashtags_count)


Distinct hashtags count: 1215


In [0]:
# Using spark.sql and the temporay table.

# Get the count of distinct hashtags
sql_query = """
    SELECT COUNT(DISTINCT hashtag) AS distinct_hashtags_count
    FROM (
        SELECT explode(hashtags) AS hashtag
        FROM twitter
    ) temp
"""

# Execute the SQL query
distinct_hashtags_count_df = spark.sql(sql_query).collect()[0]['distinct_hashtags_count']

# Show the count of distinct hashtags
print("Distinct hashtags count:", distinct_hashtags_count_df)

Distinct hashtags count: 1215


#### 2.6 Get top 5 hashtags

**My reasoning**: from hastaghs column, first count each hashtag how many times it appears, then order them by descending order of the this count and get the top 5 entries.
Remember that hashtags column its still in a different format than the rest of the columns.

In [0]:
# Using RDD (4 points)

# Get the count of each hashtag
hashtag_counts = rdd_twitter.flatMap(lambda x: x[2]).map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)

# Get the top 5 hashtags
top_5_hashtags = hashtag_counts.takeOrdered(5, key=lambda x: -x[1])

print("Top 5 hashtags:")
for hashtag, count in top_5_hashtags:
    print(hashtag, ":", count)


Top 5 hashtags:
طبق_القدرات_للثانويه_ياريس : 385
Corona : 319
OilPrice : 251
COVID19 : 125
corona : 123


In [0]:
# Using DataFrame (2 points)

# Explode the hashtags column to create one row per hashtag
df_exploded = df_twitter.select(explode(col('hashtags'))).groupby('col').count().sort("count", ascending = False).show(5)
df_exploded


+--------------------+-----+
|                 col|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+
only showing top 5 rows



In [0]:
# Using spark.sql and the temporay table. (2 points)

sql_query = """
    SELECT hashtag, COUNT(*) AS hashtag_count
    FROM (
        SELECT explode(hashtags) AS hashtag
        FROM twitter
    ) temp
    GROUP BY hashtag
    ORDER BY hashtag_count DESC
    LIMIT 5
"""


top5_hashtags_count_df = spark.sql(sql_query)

top5_hashtags_count_df.show()

+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|طبق_القدرات_للثان...|          385|
|              Corona|          319|
|            OilPrice|          251|
|             COVID19|          125|
|              corona|          123|
+--------------------+-------------+



#### 2.7 Get total number of tweets which are retweeted more than 100 times

**My reasoning**: "retweet column" (8) corresponds to the number of times each tweet has been retweeted. From this column, select (filter) those entries haigher than 100, count how many fulfill this condition.

In [0]:
# Using RDD

# knowing retweeted count is column 8
# Filter tweets with retweet count more than 100
retweeted_tweets = rdd_twitter.filter(lambda tweet: tweet[8] > 100)

# Count the total number of retweeted tweets
total_retweeted_tweets = retweeted_tweets.count()

print("Total number of tweets retweeted more than 100 times:", total_retweeted_tweets)



Total number of tweets retweeted more than 100 times: 15753


In [0]:
# Using DataFrame
from pyspark.sql.functions import col

# Filter tweets with retweet count more than 100
retweeted_tweets_df = df_twitter.filter(col("retweet_count") > 100)

# Count the total number of retweeted tweets
total_retweeted_tweets_df = retweeted_tweets_df.count()

print("Total number of tweets retweeted more than 100 times:", total_retweeted_tweets_df)


Total number of tweets retweeted more than 100 times: 15753


In [0]:
# Using spark.sql and the temporay table.
sql_query = """
    SELECT COUNT(*) AS retweet_count_above_100
    FROM twitter
    WHERE retweet_count > 100
"""

# Execute the SQL query
retweet_count_df = spark.sql(sql_query)

# Show the result
print("Total number of tweets retweeted more than 100 times:", retweet_count_df.collect()[0]['retweet_count_above_100'])

Total number of tweets retweeted more than 100 times: 15753


#### 2.8 Get top 3 most retweeted tweets per country (8 points)

**My reasoning**: From the 'retweeted tweets' (8) column and 'location' column (3) group tweets x country, from this selection, get the descending orfer and select the top 3 tweets for each group (country). I will also  print the retweeted column for each selected tweet in order to ensure that I'm doing the correct search.

In [0]:
# Using RDD

# Map to extract the required fields (country, tweet id, retweet count)
rdd_mapped = rdd_twitter.map(lambda row: (row[6], (row[3], row[8])))

# Group by country
rdd_grouped = rdd_mapped.groupByKey()

# Sort the tweets within each group by retweet count in descending order and take the top 3
rdd_top3_per_country = rdd_grouped.mapValues(lambda tweets: sorted(tweets, key=lambda x: x[1], reverse=True)[:3])

# Collect the results
top3_per_country = rdd_top3_per_country.collect()

# Print the results
for country, top3_tweets in top3_per_country:
    print(f"Country: {country}")
    for tweet in top3_tweets:
        print(f"    Tweet ID: {tweet[0]}, Retweet Count: {tweet[1]}")

Country: India
    Tweet ID: 1252332114948874240, Retweet Count: 9988
    Tweet ID: 1252252336921206787, Retweet Count: 9976
    Tweet ID: 1252254519116746754, Retweet Count: 9973
Country: Pakistan
    Tweet ID: 1252334264248606720, Retweet Count: 9988
    Tweet ID: 1252251912084357121, Retweet Count: 9975
    Tweet ID: 1252252126694309888, Retweet Count: 9973
Country: USA
    Tweet ID: 1252331777806524416, Retweet Count: 9994
    Tweet ID: 1252254239805579264, Retweet Count: 9987
    Tweet ID: 1252335464750735362, Retweet Count: 9982
Country: Italy
    Tweet ID: 1252252106750377996, Retweet Count: 9994
    Tweet ID: 1252251206027816960, Retweet Count: 9984
    Tweet ID: 1252330500670337024, Retweet Count: 9971
Country: Canada
    Tweet ID: 1252335430323888128, Retweet Count: 9997
    Tweet ID: 1252254877939531776, Retweet Count: 9992
    Tweet ID: 1252252082825986051, Retweet Count: 9987
Country: China
    Tweet ID: 1252335780707684352, Retweet Count: 9998
    Tweet ID: 12522535965168

In [0]:
# Using DataFrame

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window


# Filter out rows where the location is None (not really necessary)
df_filtered = df_twitter.filter(col("location").isNotNull())

# Define window specification (for each location order descending retweet count)
window_spec = Window.partitionBy("location").orderBy(col("retweet_count").desc())

# Add row number within each partition (country)
df_with_row_num = df_filtered.withColumn("row_number", row_number().over(window_spec))

# Filter to get only the top 3 retweeted tweets per country
top3_per_country_df = df_with_row_num.filter(col("row_number") <= 3).drop("row_number")

# Show the result
top3_per_country_df.select("location", "id", "retweet_count").show(truncate=False)

+--------+-------------------+-------------+
|location|id                 |retweet_count|
+--------+-------------------+-------------+
|Canada  |1252335430323888128|9997         |
|Canada  |1252254877939531776|9992         |
|Canada  |1252252082825986051|9987         |
|Chile   |1252253612140490759|9988         |
|Chile   |1252334891951427585|9984         |
|Chile   |1252253710182481920|9978         |
|China   |1252335780707684352|9998         |
|China   |1252253596516843520|9993         |
|China   |1252255562525560832|9984         |
|Germany |1252334028092399622|9999         |
|Germany |1252330902325248000|9997         |
|Germany |1252252295510855682|9990         |
|India   |1252332114948874240|9988         |
|India   |1252252336921206787|9976         |
|India   |1252254519116746754|9973         |
|Italy   |1252252106750377996|9994         |
|Italy   |1252251206027816960|9984         |
|Italy   |1252330500670337024|9971         |
|Mexico  |1252253843145912320|9998         |
|Mexico  |

In [0]:
# Using spark.sql and the temporay table.

# Get the top 3 most retweeted tweets per country
sql_query = """
    SELECT country, id, retweet_count
    FROM (
        SELECT location AS country, id, retweet_count,
               ROW_NUMBER() OVER (PARTITION BY location ORDER BY retweet_count DESC) AS rank
        FROM twitter
        WHERE location IS NOT NULL
    ) ranked
    WHERE rank <= 3
    ORDER BY country, retweet_count DESC
"""

# Execute the SQL query
top_retweeted_tweets_df = spark.sql(sql_query)

top_retweeted_tweets_df.show()

+-------+-------------------+-------------+
|country|                 id|retweet_count|
+-------+-------------------+-------------+
| Canada|1252335430323888128|         9997|
| Canada|1252254877939531776|         9992|
| Canada|1252252082825986051|         9987|
|  Chile|1252253612140490759|         9988|
|  Chile|1252334891951427585|         9984|
|  Chile|1252253710182481920|         9978|
|  China|1252335780707684352|         9998|
|  China|1252253596516843520|         9993|
|  China|1252255562525560832|         9984|
|Germany|1252334028092399622|         9999|
|Germany|1252330902325248000|         9997|
|Germany|1252252295510855682|         9990|
|  India|1252332114948874240|         9988|
|  India|1252252336921206787|         9976|
|  India|1252254519116746754|         9973|
|  Italy|1252252106750377996|         9994|
|  Italy|1252251206027816960|         9984|
|  Italy|1252330500670337024|         9971|
| Mexico|1252253843145912320|         9998|
| Mexico|1252255209776189442|   

#### 2.9 Total number of tweets per country

**My reasoning**: Having the groups of tweets x countries (similar as in previous exercise), count how many tweets (not retweets) are for each country.

In [0]:
# Using RDD (3 points)

# Map, filter, and reduce to get the total number of tweets per country
country_tweet_counts = (
    rdd_twitter.map(lambda row: (row[6], 1))  # Map each location to a count of 1
    .filter(lambda x: x[0] is not None)     # Filter out rows with null locations (not really necessary)
    .reduceByKey(lambda a, b: a + b)        # Reduce by key (country) to sum the counts
    .collect()                              # Collect the results
)

# Print the results
for country, count in country_tweet_counts:
    print(f"{country}: {count} tweets")

India: 1480 tweets
Pakistan: 1470 tweets
USA: 1539 tweets
Italy: 1422 tweets
Canada: 1441 tweets
China: 1457 tweets
Chile: 1410 tweets
UK: 1376 tweets
Mexico: 1409 tweets
Spain: 1464 tweets
Germany: 1426 tweets


In [0]:
# Using DataFrame (2 points)
# Group by country and count the number of tweets
tweets_per_country_df = df_twitter.groupBy("location").count()

tweets_per_country_df.show(truncate=False)

+--------+-----+
|location|count|
+--------+-----+
|Germany |1426 |
|India   |1480 |
|China   |1457 |
|Chile   |1410 |
|Italy   |1422 |
|Spain   |1464 |
|USA     |1539 |
|Mexico  |1409 |
|UK      |1376 |
|Canada  |1441 |
|Pakistan|1470 |
+--------+-----+



In [0]:
# Using spark.sql and the temporay table. (1 point)

sql_query = """
    SELECT location AS country, COUNT(*) AS total_tweets
    FROM twitter
    WHERE location IS NOT NULL
    GROUP BY location
"""

# Execute the SQL query
tweets_per_country_sql_df = spark.sql(sql_query)

tweets_per_country_sql_df.show(truncate=False)

+--------+------------+
|country |total_tweets|
+--------+------------+
|Germany |1426        |
|India   |1480        |
|China   |1457        |
|Chile   |1410        |
|Italy   |1422        |
|Spain   |1464        |
|USA     |1539        |
|Mexico  |1409        |
|UK      |1376        |
|Canada  |1441        |
|Pakistan|1470        |
+--------+------------+



## 3. Save Data

#### 3.1 save the data such that you have seperate folder per country (2 points)

In [0]:
# Using DataFrame

# Column `hashtags` has a data type of array<string>, which is not supported by CSV.
from pyspark.sql.functions import concat_ws

# Convert the array of hashtags into a single string with commas
df_twitter = df_twitter.withColumn("hashtags", concat_ws(",", "hashtags"))

# divide the df per country ('location') and save it as a .csv
df_twitter.write.partitionBy('location').csv("locations", header = 'true')

#### 3.2 Save the data as parquet files (1 points)

In [0]:
# Using DataFrame
df_twitter.write.partitionBy('location').parquet('/FileStore/tables/outputs')